# import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns' ,500)

In [30]:
"""
Importing training data and test data 
And importing training and testing with descriptors
"""
train_data = pd.read_csv("./data_train.csv")
test_data = pd.read_csv("./data_test.csv")
featured_train_data = pd.read_csv("./train_with_description.csv")
featured_test_data = pd.read_csv("./test_with_description.csv")

In [31]:
# Adding assay id to featured train data and test data
def split_data_into_assay_id_and_compound(data,column):
    data1 = []
    data2 = []
    for column in data[column]:
        split_data = column.split(";")
        chemical = split_data[0]
        assay_id = int(split_data[-1])
        data1.append(chemical)
        data2.append(assay_id)
    data["Chemical"] = data1
    data["assay_id"] = data2
    return data;

In [32]:
# Splitting data into assay Id and chemical 
train_data = split_data_into_assay_id_and_compound(train_data,"Id")
test_data = split_data_into_assay_id_and_compound(test_data,"x")

In [33]:
# Adding Assay Id to training data and testing data
featured_train_data["assay_id"] = train_data["assay_id"]
featured_train_data["Expected"] = train_data["Expected"]
featured_test_data["assay_id"] = test_data["assay_id"]

In [34]:
featured_train_data

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,MaxPartialCharge,MinPartialCharge,MaxAbsPartialCharge,MinAbsPartialCharge,FpDensityMorgan1,FpDensityMorgan2,FpDensityMorgan3,BCUT2D_MWHI,BCUT2D_MWLOW,BCUT2D_CHGHI,BCUT2D_CHGLO,BCUT2D_LOGPHI,BCUT2D_LOGPLOW,BCUT2D_MRHI,BCUT2D_MRLOW,BalabanJ,BertzCT,Chi0,Chi0n,Chi0v,Chi1,Chi1n,Chi1v,Chi2n,Chi2v,Chi3n,Chi3v,Chi4n,Chi4v,HallKierAlpha,Ipc,Kappa1,Kappa2,Kappa3,LabuteASA,PEOE_VSA1,PEOE_VSA10,PEOE_VSA11,PEOE_VSA12,PEOE_VSA13,PEOE_VSA14,PEOE_VSA2,PEOE_VSA3,PEOE_VSA4,PEOE_VSA5,PEOE_VSA6,PEOE_VSA7,PEOE_VSA8,PEOE_VSA9,SMR_VSA1,SMR_VSA10,SMR_VSA2,SMR_VSA3,SMR_VSA4,SMR_VSA5,SMR_VSA6,SMR_VSA7,SMR_VSA8,SMR_VSA9,SlogP_VSA1,SlogP_VSA10,SlogP_VSA11,SlogP_VSA12,SlogP_VSA2,SlogP_VSA3,SlogP_VSA4,SlogP_VSA5,SlogP_VSA6,SlogP_VSA7,SlogP_VSA8,SlogP_VSA9,TPSA,EState_VSA1,EState_VSA10,EState_VSA11,EState_VSA2,EState_VSA3,EState_VSA4,EState_VSA5,EState_VSA6,EState_VSA7,EState_VSA8,EState_VSA9,VSA_EState1,VSA_EState10,VSA_EState2,VSA_EState3,VSA_EState4,VSA_EState5,VSA_EState6,VSA_EState7,VSA_EState8,VSA_EState9,FractionCSP3,HeavyAtomCount,NHOHCount,NOCount,NumAliphaticCarbocycles,NumAliphaticHeterocycles,NumAliphaticRings,NumAromaticCarbocycles,NumAromaticHeterocycles,NumAromaticRings,NumHAcceptors,NumHDonors,NumHeteroatoms,NumRotatableBonds,NumSaturatedCarbocycles,NumSaturatedHeterocycles,NumSaturatedRings,RingCount,MolLogP,MolMR,fr_Al_COO,fr_Al_OH,fr_Al_OH_noTert,fr_ArN,fr_Ar_COO,fr_Ar_N,fr_Ar_NH,fr_Ar_OH,fr_COO,fr_COO2,fr_C_O,fr_C_O_noCOO,fr_C_S,fr_HOCCN,fr_Imine,fr_NH0,fr_NH1,fr_NH2,fr_N_O,fr_Ndealkylation1,fr_Ndealkylation2,fr_Nhpyrrole,fr_SH,fr_aldehyde,fr_alkyl_carbamate,fr_alkyl_halide,fr_allylic_oxid,fr_amide,fr_amidine,fr_aniline,fr_aryl_methyl,fr_azide,fr_azo,fr_barbitur,fr_benzene,fr_benzodiazepine,fr_bicyclic,fr_diazo,fr_dihydropyridine,fr_epoxide,fr_ester,fr_ether,fr_furan,fr_guanido,fr_halogen,fr_hdrzine,fr_hdrzone,fr_imidazole,fr_imide,fr_isocyan,fr_isothiocyan,fr_ketone,fr_ketone_Topliss,fr_lactam,fr_lactone,fr_methoxy,fr_morpholine,fr_nitrile,fr_nitro,fr_nitro_arom,fr_nitro_arom_nonortho,fr_nitroso,fr_oxazole,fr_oxime,fr_para_hydroxylation,fr_phenol,fr_phenol_noOrthoHbond,fr_phos_acid,fr_phos_ester,fr_piperdine,fr_piperzine,fr_priamide,fr_prisulfonamd,fr_pyridine,fr_quatN,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea,assay_id,Expected
0,9.316200,-1.533785,9.316200,0.150485,0.794714,317.599,306.511,315.982463,100.0,0.0,0.200859,-0.507967,0.507967,0.200859,0.684211,1.000000,1.263158,35.582517,9.922283,2.346931,-2.250979,2.455371,-2.178920,6.678262,0.474593,2.474117,498.898914,14.043606,9.724473,11.992260,8.876029,5.522920,6.656814,4.157690,6.526629,2.780170,3.434823,1.903958,2.659887,-1.09,1.538606e+04,14.321871,5.494957,3.470908,125.716648,10.213055,11.499024,0.000000,3.792536,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,59.068288,35.392371,0.000000,5.917906,10.213055,34.802820,0.0,0.000000,0.000000,9.710442,0.000000,59.657840,0.0,11.499024,0.000000,0.000000,11.499024,34.802820,14.005591,0.000000,0.000000,17.044809,48.530937,0.000000,0.000000,0.0,40.46,9.710442,10.213055,0.0,11.499024,0.000000,11.126903,0.000000,48.530937,0.000000,0.000000,34.802820,-1.533785,18.161033,0.000000,18.632400,1.534240,-0.186345,12.975791,0.000000,0.000000,0.000000,0.142857,19.0,2.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,2.0,2.0,5.0,2.0,0.0,0.0,0.0,2.0,4.59990,78.3466,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1644,2
1,10.532611,0.333788,10.532611,0.333788,0.516641,156.269,136.109,156.151415,66.0,0.0,0.129340,-0.300031,0.300031,0.129340,1.000000,1.5454

In [35]:
def ones_twos(x):
    X = [np.count_nonzero(x==1),np.count_nonzero(x==2)]
    print("Number of ones",np.count_nonzero(x==1))
    print("Number of twos",np.count_nonzero(x==2))

In [36]:
# Dropping null values in training data 
featured_train_data = featured_train_data.fillna(0)
featured_test_data = featured_test_data.fillna(0)

In [37]:
"""
Splitting data into X and y taking all columns 
"""
X = featured_train_data.drop("Expected",axis=1)
y = featured_train_data["Expected"]

In [38]:
"""
Splitting data into training and testing 
"""
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25, random_state=25)

In [39]:
"""
Importing XGB classifier with parameter tuning n_estimators 
"""
from xgboost import XGBClassifier
model = XGBClassifier(random_state=42,n_estimators=400,eta=0.3)

In [40]:
"""
Perform label encoding to fit in XGB classifier
"""
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test  = le.fit_transform(y_test)
y = le.fit_transform(y)

In [25]:
"""
Training mode
"""
model.fit(X_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eta=0.35, eval_metric=None,
              feature_types=None, gamma=None, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=900, n_jobs=None,
              num_parallel_tree=None, predictor=None, ...)

In [14]:
"""
Checking cross validation score using f1 and k fold validator 
"""
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model, X, y,scoring="f1", cv=5).mean()
print("My Local F1 score",scores)

My Local F1 score 0.9488199718231975


In [41]:
# Fitting model with best accuracy
model.fit(X,y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eta=0.3, eval_metric=None,
              feature_types=None, gamma=None, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=400, n_jobs=None,
              num_parallel_tree=None, predictor=None, ...)

In [42]:
# Making submission 
sub = pd.read_csv("./sample_submission.csv")

In [43]:
predict = le.inverse_transform(model.predict(featured_test_data))

In [28]:
sub["Predicted"] = predict
sub.to_csv("XBG_sub10.csv",index=False)

In [27]:
ones_twos(predict)

Number of ones 1292
Number of twos 9702
